In [1]:
# Import libraries
import pandas as pd
import requests
import random
import time
from datetime import datetime

# Parameters
GroupID = ['1846', '1789']
brands = ['HP']

def retrieve_product_ids(id):
    base_url = "https://tiki.vn/api/personalish/v1/blocks/listings"
    PARAMS = {"category": id, "page": 1}
    response = requests.get(base_url, headers=HEADERS, params=PARAMS)
    time.sleep(random.uniform(3.2, 8.7))
    data = response.json()
    total_page = data["paging"]["last_page"]
    product_data = []
    for page in range(1, total_page + 1):
        PARAMS = {"category": id, "page": page}
        response = requests.get(base_url, headers=HEADERS, params=PARAMS)
        time.sleep(random.uniform(3.2, 8.7))
        data = response.json()
        for item in data["data"]:
            product_id = item["id"]
            brand_name = item.get("brand_name", None)
            product_data.append({"sub_category_id": id, "product_id": product_id, "brand_name": brand_name})
    return product_data

# Set up header
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://tiki.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

In [2]:
sub_category_df = pd.read_csv("sub_category.csv")
sub_category_df = pd.DataFrame(sub_category_df)

In [3]:
# EXTRACT product ids
product_ids = []
for sub_category_id in sub_category_df["SubCategoryID"]:
    product_data = retrieve_product_ids(sub_category_id)
    product_ids.extend(product_data)

print(f"Success fetching data for {len(product_ids)} product ids")
reference_product = pd.DataFrame(product_ids)

# Create ReferenceID by combining SubCategoryID and ProductID
reference_product["ReferenceID"] = reference_product["sub_category_id"].astype(str) + reference_product["product_id"].astype(str)

# Reorder columns
reference_product = reference_product[["ReferenceID", "sub_category_id", "product_id", "brand_name"]]

# Rename columns
reference_product.columns = ["ReferenceID", "SubCategoryID", "ProductID", "BrandName"]

# Handle dupliate records
print("Number of records in reference_product.csv: ", reference_product.shape[0])
# Remove duplicate records
print("Number of duplicate records: ", reference_product.duplicated().sum())
# Show duplicate records
duplicate_records = reference_product[reference_product.duplicated()]
print(duplicate_records)
reference_product = reference_product.drop_duplicates()
print("Number of records after removing duplicates: ", reference_product.shape[0])

# Save to csv
reference_product.to_csv('reference_product.csv', index=False, encoding='utf-8-sig')

Success fetching data for 22997 product ids
